## Overview ##
---
In this notebook, you will learn how to order a [Planet Basemap](https://developers.planet.com/docs/data/visual-basemaps/) using your [Area of Interest](https://developers.planet.com/apis/orders/basemaps/#order-basemaps-by-area-of-interest-aoi) (AOI) and a [Quad ID](https://developers.planet.com/apis/orders/basemaps/#order-basemaps-by-quad-ids-and-deliver-to-cloud). We will place this order via Planet's [Orders API](https://developers.planet.com/apis/orders/) using our the Python Requests Library

First, you'll learn how to get a Basemap ID using either [Planet Explorer](https://developers.planet.com/docs/apps/explorer/) or the [Basemap Viewer](https://www.planet.com/basemaps/#/mosaic/). Then, you'll set up a JSON order packet, set up a session with the Python Requests Library, and [deliver]((https://developers.planet.com/apis/orders/delivery/)) the Basemap to a Google Cloud Storage bucket. 
Next, we'll order another Basemap using quad IDs.


## Order a Basemap using an AOI
---
### First, find a Basemap to order
You can get a Basemap ID from Planet Explorer or the Basemap Viewer.

### Get a basemap ID from Planet Explorer
You&#8217;ll need a basemap ID to identify what basemap to download.

To pick a basemap, you can open <a href="https://www.planet.com/explorer/#" target="_blank">Planet Explorer</a> (you can learn more about Planet Explorer [here](https://developers.planet.com/docs/apps/explorer/)) and select a basemap from the dropdown menu in the lower left corner. (If you don't have access to basemaps, you'll see a message confirming in the lower right corner. Select "Get Access" to sign up for Basemaps or speak to your customer service manager.)

Once you've selected a Basemap, the Explorer updates to include that basemap in the view.

Select the search icon and time cadence, for example monthly or quarterly. Then select one of the resulting basemap IDs, such as "global_monthly_2022_05_mosaic." That's the value you'll be passing into the Orders API. 

### Get a basemap ID from the Basemap Viewer

To pick a basemap, you can open the [Basemap Viewer](https://www.planet.com/basemaps/#/mosaic/) (you can learn more about the Basemap Viewer [here](https://developers.planet.com/docs/apps/basemapsviewer/)) and select a basemap from the left sidebar menu, using the filter to narrow down the basemap you want. 

Once you've selected a basemap, and selected the right arrow >, the basemap ID displays, such as "global_monthly_2022_01_mosaic." That's the value you'll be passing into the Orders API.

<a name="how-to-get-started"></a> 
## Start coding

---

Make sure you have Planet's Python package properly downloaded. You can find out more about this [here](https://developers.planet.com/docs/pythonclient/). Find your [API key](https://developers.planet.com/quickstart/apis/).

Next set up a session by importing needed Python packages, pulling in your API Key, and make an initial request (to retrieve the Orders API parameters) to confirm a connection with the server.

In [ ]:
import os
import requests
from requests.auth import HTTPBasicAuth
import json
import time
import math

<a name="setup-session"></a> 
## Setup Session Using Orders API
---
The first step is to setup a session to access the Orders API. The Orders API can be used to download full mosaics, or to download individual parts of the mosaic known as quads. 

### Create Session to Access API
---
Use your Planet API key for successful authentication. Here, the API key is saved to a .zsh file and imported into the Python script. You can also save your key in an environment variable. 

In [ ]:
PL_API_KEY = os.environ.get('PL_API_KEY')
GCP_CREDENTIALS = os.environ.get('GCP_CREDENTIALS')
orders_url = 'https://api.planet.com/compute/ops/orders/v2'
auth = HTTPBasicAuth(PL_API_KEY, '')

# Setup the session
session = requests.Session()

# Authenticate
session.auth = (PL_API_KEY, '')

### Create an order packet
---
Package up the details of your order in a [JSON object](https://developers.planet.com/apis/orders/basemaps/#example-order-query-json-block) and make a POST request, passing in the Orders URL, your JSON, your API key, and the content-type header. We are [delivering](https://developers.planet.com/apis/orders/delivery/) this order to a Google Cloud Storage bucket. In this request, we are also using the tools [merge](https://developers.planet.com/apis/orders/tools/#merge) and [clip](https://developers.planet.com/apis/orders/tools/#clip). 

In [ ]:
order_params = {
    "name": "basemap order with geometry",
    "source_type": "basemaps",
    "order_type": "partial",
    "products": [
        {
            "mosaic_name": "global_monthly_2022_01_mosaic",
            "geometry": {
                "type": "Polygon",
                "coordinates": [
                   [
                       [4.607406, 52.353994],
                       [4.680005, 52.353994],
                       [4.680005, 52.395523],
                       [4.607406, 52.395523],
                       [4.607406, 52.353994]
                   ]
                ]
            }
        }
    ],
    "tools": [
        {"merge": {}},
        {"clip": {}}


    ],
    "delivery": {
        "google_cloud_storage": {
            "bucket": "devrel-notebooks",
            "credentials": GCP_CREDENTIALS,
            "path_prefix": "basemaps-to-cloud/"
        }
    }
}

### Print the response object
---
Retrieve the response object as text to print out if there are no errors.

In [ ]:
ORDERS_API_URL = 'https://api.planet.com/compute/ops/orders/v2'
headers = {'content-type': 'application/json'}
param_req = requests.post(ORDERS_API_URL, data=json.dumps(
    order_params), auth=auth, headers={'content-type': 'application/json'})

print(param_req.text)

### Place the order
---
The following code places the order.

In [ ]:
def place_order(order_params, auth):
    '''Place an order and return the order URL

    Parameters: order_params (dict): order parameters
                auth (requests.auth.HTTPBasicAuth): authentication
    Return (str): URL of the order
    '''
    response = requests.post(ORDERS_API_URL, data=json.dumps(
        order_params), auth=auth, headers=headers)
    order_id = response.json()['id']
    order_url = ORDERS_API_URL + '/' + order_id
    return order_url

In [ ]:
order_url = place_order(order_params, session.auth)

### Poll for data success
---
When you're ready to pull the data, you can poll for when it's ready. Here, the polling is limited to a number of loops routine.

In [ ]:
def poll_for_success(order_url, num_loops=30):
    '''Polls for order success, fails after num_loops

    Parameters: order_url (str): url for order
                num_loops (int): number of times to poll before failing
    Prints the state of the order
    '''
    count = 0
    end_states = ['success', 'failed', 'partial']
    while (count < num_loops):
        count += 1
        r = requests.get(order_url, auth=session.auth)
        response = r.json()
        state = response['state']
        print(state)
        if state in end_states:
            break
        time.sleep(10)

In [ ]:
poll_for_success(order_url)

In [ ]:
r = requests.get(order_url, auth=session.auth)
response = r.json()
results = response['_links']['results']

print([r['name'] for r in results])

## Use Basemaps API to get the Basemap and quad IDs
---

Use the Basemap APIs to retrieve the basemap you want and the quad IDs. You can get the base URL you'll need to communicate with the Basemap API service:

1. Go to the [Basemap API reference](https://developers.planet.com/docs/basemaps/reference/) at https://developers.planet.com/docs/basemaps/reference/. 
2. Under List Mosaics, select GET /mosaics.

A dropdown UI element appears with the URL to use:

https://api.planet.com/basemaps/v1/mosaics


#### Search for the mosaic ID to get its quad IDs

Here, we are using the full name of the mosaic, but with the `name__contains` field, you can use a partial name based on the names of the mosaics available through your account.

Declare the search string for `requests.get()`. You will use the dictionary you create with the mosaic name to pass to the requests.get() function.


This code sets the URL and headers for an HTTP request to Planet's Basemap API and sends a GET request with the specified search parameters. If the response returns the correct status, the JSON response is loaded into a Python dictionary with mosaics information from which we'll extract quad ID information.

In [ ]:
BASEMAP_API_URL = 'https://api.planet.com/basemaps/v1/mosaics'
headers = {'content-type': 'application/json'}

basemap_params = {
    'name__contains': 'point_reyes_rolling_normalized_2020-04-30_mosaic'
}

basemap_service_response = requests.get(
    url=BASEMAP_API_URL, params=basemap_params, auth=auth, headers=headers)

if basemap_service_response.status_code == 204:
    basemaps = basemap_service_response.raise_for_status()
else:
    basemaps = json.loads(basemap_service_response.text)

This code retrieves the ID and bounding box coordinates of the mosaic we loaded above and uses these to construct an API request to retrieve the corresponding quad IDs for a specific area.

In [ ]:
mosaic_id = basemaps['mosaics'][0]['id']
mosaic_bbox = basemaps['mosaics'][0]['bbox']
mosaic_bbox_str = ','.join(map(str, mosaic_bbox))

quad_params = {
    'mosaic_id': mosaic_id,
    'bbox': mosaic_bbox_str,
}

quads_url = "{}/{}/quads".format(BASEMAP_API_URL, mosaic_id)
quad_service_response = requests.get(
    url=quads_url, params=quad_params, auth=auth, headers=headers)
quads = quad_service_response.json()
items = quads['items']

quad_ids = [i['id'] for i in items]

### Create an order packet
---
Package up the details of your order in a [JSON object](https://developers.planet.com/apis/orders/basemaps/#example-order-query-json-block) and make a POST request, passing in the Orders URL, your JSON, your API key, and the content-type header. We are [delivering](https://developers.planet.com/apis/orders/delivery/) this order to a Google Cloud Storage bucket. In this example, we are using the tools [bandmath](https://developers.planet.com/apis/orders/tools/#band-math) and [reproject](https://developers.planet.com/apis/orders/tools/#reproject).

#### Reproject

Reprojecting a basemap is necessary when the original map projection does not accurately represent the Earth's surface in a specific region or area. This can happen when the imagery for the basemap comes from a place far from the equator. In the example shown in this notebook, there is not a significant change in circumference of the earth from the equator, but it is always best to check using the code below. 

First, find the latitude of the area of interest. Then, find the circumference of the Earth at the given latitude in kilometers. We recommend using a website like [this](http://www.easysurf.cc/circle.htm#cetol1) Use the 'Calculate Circumference of the Earth at a given Latitude' calculator. Enter the latitude of the area you are looking for. In the code below, replace the parameters with your latitude and the circumference of the earth in kilometers. 

In [ ]:
def calculate_resolution(latitude):
    '''Calculate the resolution of a basemap at a given latitude

    Parameters: latitude (float): latitude at location of interest [degrees]
    Return (float): resolution of basemap at given latitude [degrees per pixel]
    '''
    earth_radius = 6378137.0  # meters
    native_resolution = 3  # meters per pixel
    circumference_of_earth = 2.0 * math.pi * (earth_radius)
    meters_per_degree_at_latitude = math.cos(math.radians(
        latitude)) * circumference_of_earth / 360.0
    resolution_at_latitude = native_resolution / meters_per_degree_at_latitude
    return resolution_at_latitude

In [ ]:
resolution_at_latitude = calculate_resolution(37.0)

In [ ]:
order_params = {
    "name": "basemap order with quad_ids",
    "source_type": "basemaps",
    "products": [
        {
            "mosaic_name": "point_reyes_rolling_normalized_2020-04-30_mosaic",
            "quad_ids": quad_ids
        }
    ],
    "tools": [
        {
            "reproject": {
                "projection": "EPSG:4326",
                "resolution": resolution_at_latitude,
                "kernel": "cubic"
            }
        },
        {
            "bandmath": {
                "b1": "b1",
                "b2": "b2",
                "b3": "b3",
                "b4": "arctan(b1)",
                "b5": "(b4-b3)/(b4+b3)",
                "pixel_type": "32R"
            }
        }
    ],
    "delivery": {
        "google_cloud_storage": {
            "bucket": "devrel-notebooks",
            "credentials": GCP_CREDENTIALS,
            "path_prefix": "basemaps-to-cloud/",
        }
    }
}

### Send Request and Check Status Code
---
When sending the request, we will provide one parameter. Assuming we know the name of the mosaic, we can include those details in the request to access the corresponding metadata. After sending the request, let's check the status code to ensure it was successful.

Send a get request and print out the response, which should return `<Response [202]>`.

In [ ]:
param_res = requests.post(ORDERS_API_URL, data=json.dumps(
    order_params), auth=auth, headers=headers)

if param_res.status_code == 204:
    print(param_res.raise_for_status())
else:
    print(param_res.text)

In [ ]:
r = requests.get(order_url, auth=session.auth)
response = r.json()
results = response['_links']['results']
print([r['name'] for r in results])

### Check your results 

After a few simple steps, we have basemaps ordered to our Google Cloud Platform bucket using the Orders API and Python Requests Library. For more information, check out the docs [here](https://developers.planet.com/apis/orders/). 
